## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [4]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set

## Loading and preparing training data
The train and test images are given in two different folders - 'train' and 'test'. The labels of train images are given in a csv file 'Train.csv' with respective image id (i.e. image file name).

#### Getting the labels of the images

In [5]:
labels = pd.read_csv("Training_set.csv")   # loading the labels
labels.head()           # will display the first five rows in labels dataframe

,filename,label
0,Image_1.jpg,six of clubs
1,Image_2.jpg,queen of hearts
2,Image_3.jpg,seven of diamonds
3,Image_4.jpg,six of spades
4,Image_5.jpg,eight of spades


In [6]:
labels['label'].unique()

array(['six of clubs', 'queen of hearts', 'seven of diamonds',
       'six of spades', 'eight of spades', 'king of hearts',
       'ten of spades', 'jack of diamonds', 'three of clubs',
       'queen of spades', 'jack of clubs', 'eight of clubs',
       'two of diamonds', 'queen of diamonds', 'jack of hearts',
       'seven of clubs', 'seven of spades', 'six of diamonds',
       'ten of hearts', 'nine of spades', 'seven of hearts',
       'ace of spades', 'four of diamonds', 'ten of diamonds', 'joker',
       'nine of clubs', 'five of hearts', 'three of spades',
       'four of spades', 'ten of clubs', 'king of spades',
       'two of spades', 'nine of diamonds', 'nine of hearts',
       'ace of diamonds', 'two of clubs', 'ace of clubs', 'five of clubs',
       'two of hearts', 'five of spades', 'three of diamonds',
       'eight of diamonds', 'jack of spades', 'king of diamonds',
       'queen of clubs', 'five of diamonds', 'three of hearts',
       'four of hearts', 'king of clubs', 

In [7]:
labels.tail()            # will display the last five rows in labels dataframe

,filename,label
4771,Image_4772.jpg,ace of clubs
4772,Image_4773.jpg,jack of hearts
4773,Image_4774.jpg,five of clubs
4774,Image_4775.jpg,two of clubs
4775,Image_4776.jpg,five of clubs


#### Getting images file path

In [8]:
file_paths = [[fname, '/train/' + fname] for fname in labels['filename']]

#### Confirming if no. of labels is equal to no. of images

In [9]:
# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
    print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

Number of labels i.e.  4776 matches the number of filenames i.e.  4776


#### Converting the file_paths to dataframe

In [10]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,/content/train/Image_1.jpg
1,Image_2.jpg,/content/train/Image_2.jpg
2,Image_3.jpg,/content/train/Image_3.jpg
3,Image_4.jpg,/content/train/Image_4.jpg
4,Image_5.jpg,/content/train/Image_5.jpg


#### Combining the labels with the images

In [11]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()       

,filename,filepaths,label
0,Image_1.jpg,/content/train/Image_1.jpg,six of clubs
1,Image_2.jpg,/content/train/Image_2.jpg,queen of hearts
2,Image_3.jpg,/content/train/Image_3.jpg,seven of diamonds
3,Image_4.jpg,/content/train/Image_4.jpg,six of spades
4,Image_5.jpg,/content/train/Image_5.jpg,eight of spades


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])

The 'train_data' dataframe contains all the image id, their locations and their respective labels. Now the training data is ready.

## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [13]:
print(train_data['label'][1])

31


In [14]:
data = [] # initialize an empty numpy array

image_size = 224 # image size taken is 224 here. one can take other size too
for i in range(len(train_data)):

  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_COLOR) # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size),cv2.INTER_AREA) # resizing the image array

  # update the data and labels lists, respectively
  data.append([new_img_array, train_data['label'][i]])



In [15]:
print(labels)

            filename              label
0        Image_1.jpg       six of clubs
1        Image_2.jpg    queen of hearts
2        Image_3.jpg  seven of diamonds
3        Image_4.jpg      six of spades
4        Image_5.jpg    eight of spades
...              ...                ...
4771  Image_4772.jpg       ace of clubs
4772  Image_4773.jpg     jack of hearts
4773  Image_4774.jpg      five of clubs
4774  Image_4775.jpg       two of clubs
4775  Image_4776.jpg      five of clubs

[4776 rows x 2 columns]


In [16]:
len(set(labels))

2

In [17]:
np.random.shuffle(data)

In [18]:
x = []
y = []
for image in data:
  x.append(image[0])
  y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

#### Splitting the data into Train and Validation Set
We want to check the performance of the model that we built. For this purpose, we always split (both independent and dependent data) the given data into training set which will be used to train the model, and test set which will be used to check how accurately the model is predicting outcomes.

For this purpose we have a class called 'train_test_split' in the 'sklearn.model_selection' module.

In [19]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(x, y,
	test_size=0.20,  random_state=4)

In [20]:
# building the input vector from the 28x28 pixels
X_train = trainX.reshape(trainX.shape[0], 224, 224, 3)
X_test = testX.reshape(testX.shape[0], 224, 224, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [21]:
# normalizing the data to help with the training
X_train /= 255
X_test /= 255

In [22]:

from keras.utils import np_utils

# one-hot encoding using keras' numpy-related utilities
n_classes = 53
print("Shape before one-hot encoding: ", trainY.shape)
Y_train = np_utils.to_categorical(trainY, n_classes)
Y_test = np_utils.to_categorical(testY, n_classes)
print("Shape after one-hot encoding: ", trainY.shape)

Shape before one-hot encoding:  (3820,)
Shape after one-hot encoding:  (3820,)


## Building Model
Now we are finally ready, and we can train the model.

There are many machine learning or deep learning models like Random Forest, Decision Tree, Multi-Layer Perceptron (MLP), Convolution Neural Network (CNN), etc. to say you some.


Then we would feed the model both with the data (X_train) and the answers for that data (y_train)

In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras.regularizers import l1_l2

model = Sequential()

#### Input Layer ####
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',
                 activation='relu', input_shape=(224, 224, 3)))

#### Convolutional Layers ####
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))  # Pooling
model.add(Dropout(0.2)) # Dropout

model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(512, (5,5), padding='same', activation='relu'))
model.add(Conv2D(512, (5,5), activation='relu'))
model.add(MaxPooling2D((4,4)))
model.add(Dropout(0.2))

#### Fully-Connected Layer ####
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(53, activation='softmax'))

In [24]:
from keras.optimizers import Adam
n_epochs = 20

optimizer = Adam(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [25]:
model.fit(X_train, Y_train,32,
                    steps_per_epoch=len(X_train) // 32,
	                  validation_data=(X_test, Y_test),
	                  validation_steps=len(X_test) // 32,
                    epochs=n_epochs
                    )

Epoch 1/20
119/119 [==============================] - 36s 205ms/step - loss: 3.8771 - accuracy: 0.0312 - val_loss: 3.6602 - val_accuracy: 0.0528
Epoch 2/20
119/119 [==============================] - 26s 202ms/step - loss: 3.2242 - accuracy: 0.1463 - val_loss: 2.7502 - val_accuracy: 0.2435
Epoch 3/20
119/119 [==============================] - 24s 202ms/step - loss: 2.4939 - accuracy: 0.2970 - val_loss: 2.2396 - val_accuracy: 0.3545
Epoch 4/20
119/119 [==============================] - 24s 198ms/step - loss: 2.0706 - accuracy: 0.3973 - val_loss: 2.0262 - val_accuracy: 0.4256
Epoch 5/20
119/119 [==============================] - 24s 199ms/step - loss: 1.7915 - accuracy: 0.4765 - val_loss: 1.9034 - val_accuracy: 0.4569
Epoch 6/20
119/119 [==============================] - 24s 199ms/step - loss: 1.5093 - accuracy: 0.5499 - val_loss: 1.7886 - val_accuracy: 0.4925
Epoch 7/20
119/119 [==============================] - 24s 199ms/step - loss: 1.2874 - accuracy: 0.6162 - val_loss: 1.6869 - val_ac

## Validate the model
Performance on the X_val data.

In [26]:
model.evaluate(X_test, Y_test)

30/30 [==============================] - 3s 109ms/step - loss: 2.2599 - accuracy: 0.6579


[2.259890556335449, 0.6579498052597046]

## Predict The Output For Testing Dataset
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

In [27]:
# Loading the order of the image's name that has been provided
test_image_order = pd.read_csv("Testing_set.csv")
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


#### Getting images file path

In [28]:
file_paths = [[fname, '/test/' + fname] for fname in test_image_order['filename']]

#### Confirm if number of images in test folder is same as number of image names in 'Testing_set_face_mask.csv'

In [29]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
    print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
    print('Number of image names does not match the number of filepaths')

Number of image names i.e.  2048 matches the number of file paths i.e.  2048


#### Converting the file_paths to dataframe

In [30]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.tail()

,filename,filepaths
2043,Image_2044.jpg,/content/test/Image_2044.jpg
2044,Image_2045.jpg,/content/test/Image_2045.jpg
2045,Image_2046.jpg,/content/test/Image_2046.jpg
2046,Image_2047.jpg,/content/test/Image_2047.jpg
2047,Image_2048.jpg,/content/test/Image_2048.jpg


## Data Pre-processing on test_data


In [31]:
test_pixel_data = []     # initialize an empty numpy array
image_size = 224      # image size taken is 100 here. one can take other size too
missing_files = []
for i in range(len(test_images)):
    img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_COLOR)   # converting the image to gray scale

    new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array

    test_pixel_data.append(new_img_array)


In [32]:
test_pixel_data = np.array(test_pixel_data)

In [33]:
len(test_pixel_data)

2048

### Make Prediction on Test Dataset
Time to make a submission!!!

In [34]:
pred = model.predict(test_pixel_data)

64/64 [==============================] - 4s 59ms/step


In [35]:
# The predicted values are the probabilities value
pred[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.], dtype=float32)

The above values are probability values. We need to convert it into respective classes. We can use np.argmax for the same.

In [36]:
prediction = []
for value in pred:
  prediction.append(np.argmax(value))

In [37]:
predictions = le.inverse_transform(prediction)

## **Saving prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [38]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **Save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [39]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>